In [7]:
from Bio import SeqIO
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
        
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1  

def main():
    K_index=[]
    K_index_N=[]
    for_value=[]
    all_value=[]
    truncated=[]
    full_truncated=[]
    Seq=[]
    lst=[]
    Acet=[]
    yes=[]
    
    f= open("Dataset_creation.txt","w+")
    records = list(SeqIO.parse("Acetylation_Homo.fasta.txt", "fasta"))
#     print(records)
    sequence=[]
    for seq_record in SeqIO.parse("Acetylation_Homo.fasta.txt", "fasta"):
        sequence.append(seq_record.seq)

    # print (len(sequence))
#     print(sequence)
    length=len(sequence)
    names=[]
    for seq_record in SeqIO.parse("Acetylation_Homo.fasta.txt", "fasta"):
        name=seq_record.name.split("|")[1]
        names.append(name)
    # print(names)
    names_len=len(names)
    
    
# Window creation of size 31

    for seq_record in range(0,length):
        S=list(sequence[seq_record])
        Seq.append(S)
#         print(Seq)
        occurrences = lambda s, lst: (i for i,e in enumerate(lst) if e == s)
        K = list(occurrences('K', S))
        K_new=list(map(lambda x: x + 1, K))
        K_index_N.append(K_new)
        K_index.append(K)
        
        
    K_lenth=len(K_index)                   
    for i in range(0,K_lenth):
        K_value=K_index[i]
        S_value=Seq[i]
        max_len=len(S_value)
#         print(names[i])
#         print(K_value)
        for j in K_value:
            
            if (j-15>0) and (j+15<max_len):
                X=S_value[j-15:j+16]
            elif (j-15<0) and (j+15<max_len):
                Neg=15-j
                
                X1=['X']*Neg
                X2=S_value[0:j]
                X3=list(S_value[j])
                X4=S_value[j+1:j+16]
                X=X1+X2+X3+X4
            elif (j-15>0) and (j+15>max_len):
                pos=(j+15)-(max_len-1)
                X1=S_value[j-15:j]
                X2=list(S_value[j])
                X3=S_value[j+1:max_len]
                X4=['X']*pos
                X=X1+X2+X3+X4
                
            
            Str= listToString(X)
            truncated.append(Str)
            
#             Str= listToString(Str_X)            
#             f.write(Str)
#         f.write("\n")
#         print(truncated)
        full_truncated.append(truncated)
        truncated=[]
    
    
    print("\n")
#     print(full_truncated)
#     f.close()

# Code for Yes/No positions
    df = pd.read_excel('Acetylation_Homo.xlsx', sheet_name='Sheet1')
    protein_no = list(df['Uniprot Accession'])
    value = list(df['Position'])
    protien_len=len(protein_no)

    for i in range(0,names_len):
#         print(protein_no)  
        occurrences = lambda s, lst: (k for k,e in enumerate(lst) if e == s)
#         print(names[i])
        N = list(occurrences(names[i], protein_no))
        yes_len=len(N)
        KN_len=len(K_index_N[i])
        Acet=['No']*KN_len
        for k in N:
            try:
                yes_index=K_index_N[i].index(value[k])
                yes.append(yes_index)
                Acet[yes_index]="Yes"
            except:
                print(names[i])
            
        lst.append(Acet)
#     print(lst)
    Uniprot=[]
    Site=[]
    Peptide=[]
    Acetylation=[]
# DataFrame/dataset Create                    

    for j in range(0,length):
        K_lenth=len(K_index[j])
        K_value=K_index[j]
        for i in range(0,K_lenth):
            Uniprot.append(names[j])
            
#     print(Uniprot)
            
            
    for j in range(0,length):
        K_lenth=len(K_index[j])
        K_value=K_index_N[j]
        for k in K_value:
            Site.append(k)
#     print(Site)
    
    for j in range(0,length):
        T_lenth=len(full_truncated[j])
        T_value=full_truncated[j]
        for k in T_value:
            Peptide.append(k)
#     print(Peptide)
    
    for j in range(0,length):
        L_lenth=len(lst[j])
        L_value=lst[j]
        for k in L_value:
            Acetylation.append(k)
#     print(Acetylation)
    
    raw_data ={'Uniprot Accession': Uniprot,
            'Site': Site,
            'Peptide': Peptide,
            'Acetylation?':Acetylation
            }
    dff=pd.DataFrame(raw_data,columns=['Uniprot Accession','Site','Peptide','Acetylation?'])
    dff.to_csv('/Users/urmis/Documents/Thesis/Dataset.csv',index=False)
    print(dff)

#     
#     yes=[item for item in protein_no if item not in names]
#     print(yes)
    
            

main()




       Uniprot Accession  Site                          Peptide Acetylation?
0                 O00116    59  HLLGRPREALSTNECKARRAASAATAAPTAT           No
1                 O00116    85  APTATPAAQESGTIPKKRQEVMKWNGWGYND           No
2                 O00116    86  PTATPAAQESGTIPKKRQEVMKWNGWGYNDS           No
3                 O00116    92  AQESGTIPKKRQEVMKWNGWGYNDSKFIFNK           No
4                 O00116   102  RQEVMKWNGWGYNDSKFIFNKKGQIELTGKR          Yes
...                  ...   ...                              ...          ...
281234            K7EQG2    81  FTSGLQAGLKQSHLSKAIGSSPLGSGEGLLG           No
281235            K7EQG2   109  LLGLSPGPNGHSHLLKTPLGGQKRSFAHLLP           No
281236            K7EQG2   116  PNGHSHLLKTPLGGQKRSFAHLLPSPEPSPE          Yes
281237            K7EQG2   152  QHSQGLGGHYADSYLKRKRIFXXXXXXXXXX           No
281238            K7EQG2   154  SQGLGGHYADSYLKRKRIFXXXXXXXXXXXX           No

[281239 rows x 4 columns]
